In [ ]:
from MySQL import MySQL_Doing
import pandas as pd

MySQL_Doing = MySQL_Doing()
Results = MySQL_Doing.run("show tables")
Results

In [38]:
# Station about
Results = MySQL_Doing.run("""
Select latitude, longitude , stop_name, direction 
from bus_route_stations
where route_id = '3'
""")
Results

,latitude,longitude,stop_name,direction
0,23.976161,121.608885,中華路,去程
1,23.993665,121.604872,勤天商旅,去程
2,23.993665,121.604872,勤天商旅,去程
3,23.980685,121.612726,將軍府商圈,去程
4,23.989980,121.601315,市立圖書館,去程
5,23.975530,121.606394,日出香榭大道1,去程
6,23.974270,121.607381,日出香榭大道2,去程
7,23.972523,121.609228,東大門夜市,去程
8,23.977802,121.602481,花蓮又一村文創園區,去程
9,23.975652,121.604519,花蓮文創園區,去程


In [ ]:
# Station about
Results = MySQL_Doing.run("""
SELECT 
X AS latitude,
Y AS longitude
FROM ttcarimport
WHERE car_licence = 'ABS-001';
""")
Results

In [41]:
# Station about 婦幼親創園區
Results = MySQL_Doing.run("""
select * from car_backup
""")
Results

,seq,rcv_dt,car_licence,Gpstime,X,Y,Speed,Deg,acc,route,direction,Current_Loaction
0,1,2025-09-25 20:08:22,ABS-001,2025-09-25 20:08:22,121.618438,23.988424,60,60,b'\x00',None,去程,None
1,2,2025-09-25 20:28:48,ABS-001,2025-09-25 20:28:48,121.619688,23.989522,60,60,b'\x00',1,去程,花蓮地方法院
2,3,2025-09-25 20:30:30,ABS-002,2025-09-25 20:30:30,121.622213,23.987178,60,60,b'\x00',1,回程,煙波飯店
3,4,2025-09-25 20:32:02,ABS-001,2025-09-25 20:32:02,121.619688,23.989522,60,60,b'\x00',1,去程,花蓮地方法院
4,5,2025-09-25 20:32:02,ABS-002,2025-09-25 20:32:02,121.622213,23.987178,60,60,b'\x00',2,回程,煙波飯店
5,6,2025-09-25 20:32:02,ABS-003,2025-09-25 20:32:02,121.609228,23.972523,60,60,b'\x00',3,去程,東大門夜市


In [40]:
Results = MySQL_Doing.run("""
INSERT INTO car_backup 
(car_licence, Gpstime, X, Y, Speed, Deg, acc, route, direction, Current_Loaction)
VALUES
('ABS-001', NOW(), '121.619688', '23.989522', 60, 60, b'0', '1', '去程', '花蓮地方法院');
""")

Results = MySQL_Doing.run("""
INSERT INTO car_backup 
(car_licence, Gpstime, X, Y, Speed, Deg, acc, route, direction, Current_Loaction)
VALUES
('ABS-002', NOW(), '121.622213', '23.987178', 60, 60, b'0', '2', '回程', '煙波飯店');
""")

Results = MySQL_Doing.run("""
INSERT INTO car_backup 
(car_licence, Gpstime, X, Y, Speed, Deg, acc, route, direction, Current_Loaction)
VALUES
('ABS-003', NOW(), '121.609228', '23.972523', 60, 60, b'0', '3', '去程', '東大門夜市');
""")

In [57]:
def Get_GIS_About():
    Results = MySQL_Doing.run("""
    Select route from car_resource
    where route != 'None'
    """)

    print(Results["route"].tolist())
    Results = MySQL_Doing.run("""
    SELECT c.route, c.X, c.Y, c.direction, c.Current_Loaction
    FROM car_backup c
    JOIN (
        SELECT route, MAX(seq) AS max_seq
        FROM car_backup
        WHERE route IN ('1', '2', '3')
        GROUP BY route
    ) t ON c.route = t.route AND c.seq = t.max_seq;
    """)
    return Results

Get_GIS_About()

['4', '1', '2', '3']


,route,X,Y,direction,Current_Loaction
0,1,121.619688,23.989522,去程,花蓮地方法院
1,2,121.622213,23.987178,回程,煙波飯店
2,3,121.609228,23.972523,去程,東大門夜市


In [29]:
def get_gis_about():
    # 撈出車輛基本資料
    df = pd.DataFrame(MySQL_Doing.select_all("car_resource"))
    df = df[df["route"].notnull()]   # 只留有 route 的

    # 收集車牌 (route → car_licence)
    cars = {route: car for route, car in zip(df["route"], df["car_licence"])}
    carss = list(cars.values())

    if not carss:
        return []

    # 串成 SQL IN 字串
    car_str = "', '".join(carss)

    query = f"""
    SELECT t.*
    FROM ttcarimport t
    INNER JOIN (
        SELECT car_licence, MAX(seq) AS max_seq
        FROM ttcarimport
        WHERE car_licence IN ('{car_str}')
        GROUP BY car_licence
    ) x ON t.car_licence = x.car_licence AND t.seq = x.max_seq;
    """

    df_latest = pd.DataFrame(MySQL_Doing.run(query))

    # 用 car_licence 對應回 route
    route_map = df.set_index("car_licence")["route"].to_dict()
    df_latest["route"] = df_latest["car_licence"].map(route_map)

    # pandas → dict (避免 NaN/NaT)
    records = df_latest.where(pd.notnull(df_latest), None).to_dict(orient="records")
    return records

Data = get_gis_about()
Data = pd.DataFrame(Data)
Data

,seq,rcv_dt,car_licence,Gpstime,X,Y,Speed,Deg,acc,route
0,3855,2025-09-25 16:16:34,ABS-001,2025-09-25 16:16:35,121.618438,23.988424,60,50,b'\x01',1
1,3814,2025-09-24 09:10:14,ABS-002,2025-09-24 09:10:14,23.988199,121.625951,73,168,b'\x01',2
2,3815,2025-09-24 09:10:17,ABS-003,2025-09-24 09:10:17,23.993665,121.604872,73,241,b'\x01',3
3,3816,2025-09-24 09:10:21,NND-9607,2025-09-24 09:10:21,23.980685,121.612726,50,327,b'\x01',4
